In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score as CVS
from sklearn.preprocessing import PolynomialFeatures as PF
from statsmodels.formula.api import ols
from statsmodels.tools.eval_measures import rmse as RMSE
import scipy.stats as stats
import numbers
import pickle


In [4]:
def transform(data_t, poly=None, test=False):
    """
    poly values: 'all', 'list', 'singles', 'all_single'
    Note: singles/all_single -column names only have first letters
    
    """

    #     'bathroomsx4', 'bedrooms', 'condition', 'date', 'floorsx2', 'grade', 'lat',
#        'long', 'price', 'sqft_above', 'sqft_basement', 'sqft_living',
#         'sqft_living15', 'sqft_lot', 'sqft_lot15', 'view', 'waterfront',
#         'yr_built', 'yr_renovated', 'zipcode'
    
    to_poly = ['sqft_above', 'sqft_basement', 'sqft_living',
              'sqft_living15', 'sqft_lot', 'sqft_lot15','yr_built', 
              'yr_renovated',]
    to_log = ['price', 'sqft_above', 'sqft_basement', 'sqft_living',
              'sqft_living15', 'sqft_lot', 'sqft_lot15','yr_built', 
              'yr_renovated',]
    to_norm = []#['price', 'sqft_above', 'sqft_basement', 'sqft_living',
#               'sqft_living15', 'sqft_lot', 'sqft_lot15','yr_built', 
#               'yr_renovated',]
    to_ohe = ['bedrooms', 'bathroomsx4','condition', 'floorsx2','grade','view',]
    poly_order = 2

    if poly in ['all', 'all_single']:
        to_poly = data_t.drop(to_ohe + ['price'], axis=1).columns
        if poly == 'all':
            poly = 'list'
        elif poly == 'all_single':
            poly = 'singles'
    if poly == 'list':
        poly=PF(poly_order)
        data_poly = poly.fit_transform(data_t[to_poly])
        data_poly = pd.DataFrame(data_poly)
        data_poly.columns = poly.get_feature_names(data_t[to_poly].columns)
        data_t = pd.concat([data_t.drop(to_poly, axis=1),
                            data_poly.drop('1',axis=1)], axis=1)
    elif poly == 'singles':
        for feat in to_poly:
            df = pd.DataFrame(data_t[feat])
            print(feat)
            poly=PF(poly_order)
            data_poly = poly.fit_transform(df)
            data_poly = pd.DataFrame(data_poly)
            data_poly.columns = ['drop', feat, feat + '_squared']
            data_t = pd.concat([data_t.drop(feat, axis=1), 
                                data_poly.drop('drop',axis=1)], axis=1)

    for feat in to_log:
        data_t[feat] = data_t[feat].map(lambda x: np.log(x) if x!=0 else 0)
        #Note 'if' included so that 0 values wouldn't error, will still error
        #...between 0 and 1. Also, implies that the original 0 value was a 1
        #...but as these are around values of 100s, 1000s, the effect is minimal

        
    stats = {}
    if test:
        with open('norm_stats.pickle', 'rb') as f:
            stats = pickle.load(f)
    for feat in to_norm:
        ft = data_t[feat]
        if test:
            mean = stats[feat][0]
            stdev = stats[feat][1]
        else:
            mean = np.mean(ft)
            stdev = np.sqrt(np.var(ft))
        data_t[feat] = (ft-mean) / stdev
        stats[feat] = [mean, stdev]
    with open('norm_stats.pickle', 'wb') as f:
        pickle.dump(stats, f)
        
        
    for feat in to_ohe:
        dummies = pd.get_dummies(data=data_t[feat], prefix=feat, prefix_sep='_',
                                drop_first=True)
        data_t.drop(feat, axis=1, inplace=True)
        data_t = pd.concat([data_t, dummies], axis=1)
    data_t.rename(lambda col: col.replace('.0', '').replace(' ','__')
                  .replace('^','_pow_'),
                  axis=1, inplace=True)
    
    return data_t 

#Note data_t = data.copy()   --line added as otherwise the DataFrame, data, 
#...outside of this function lost the first column mentioned in [to_ohe]

In [5]:
def simple_selector(data_s, x_cols, alpha=0.05):

    predictors = '+'.join(x_cols)
    f = 'price' + '~' + predictors
    results = ols(formula=f, data=data_s).fit()
    pv = pd.DataFrame(results.pvalues).drop('Intercept')
    pv.rename(columns={0:'p_value'}, inplace=True)
    x_cols = list(pv[pv.p_value <= alpha].index)
    
    return x_cols, pv

In [6]:
def stepwise_selector(data_s, 
                       x_cols=[], 
                       alpha=0.05, 
                       verbose=False):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        x_cols - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    
    X = data_s[x_cols]
    y = data_s['price']
    threshold_in = alpha - 0.02
    threshold_out = alpha + 0.01
    included = list(x_cols)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            results = sm.OLS(y, sm.add_constant
                             (pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = results.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        results = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = results.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, 
                                                             worst_pval))
        if not changed:
            break
            
    pv = pd.DataFrame(results.pvalues)
    pv.rename(columns={0:'p_value'}, inplace=True)
    return included, pv

## Multicollinearity Function: Remove    
The first multicollinearity function removes a feature from each pair with high multicollinearity    
This function takes in thedata, x_cols and the threshold for removing a feature

- create a dataframe of the correlation of the features in x_cols   
- transform this to get a list of pairs of features with high multicollinearity
- for each pair: check whether they are the same feature or if one of the features has already been listed to be removed; if so, continue to next pair 
- otherwise add the feature with the higher p-value to a list    
- remove the features in this list from x_cols    


return x_cols     




In [7]:
def multicoll_remove(data, x_cols, pvalues, multicollinearity_threshold):
    corr = data[x_cols].corr().abs().stack().reset_index().sort_values(0,
                                                            ascending = False)
    corr['pairs'] = list(zip(corr.level_0, corr.level_1))
    corr = corr.set_index('pairs').drop(['level_0', 'level_1'], axis=1)
    corr.columns = ['cc']
    corr = corr[corr.cc > multicollinearity_threshold]        

    to_drop = []
    for f0, f1 in corr.index:
        if (f0 == f1) | any(feat in [f0, f1] for feat in to_drop):
            continue
        to_drop.append(pvalues.loc[[f0, f1]].sort_values('p_value', 
                                                ascending=False).index[0])
    x_cols = list(set(x_cols) - set(to_drop))
    return x_cols   

In [8]:
def metrics(data_m, results, x_cols):
#     X = data[x_cols]
#     y = data.price
#     to_pred = pd.concat([y,X], axis=1)
#     yhat = results.predict(to_pred)
#     rmse = RMSE(y, yhat)
    
#     linreg = LR()
#     linreg.fit(X,y)
#     cv  = np.mean(CVS(linreg, X, y, cv=5,  scoring='neg_mean_squared_error'))
#     rmse = (-cv)**0.5
      
#     print(f'RMSE: {rmse}')
    
    fig = sm.graphics.qqplot(results.resid, dist=stats.norm, line='45',
                             fit=True, alpha=0.1)

    fig = plt.figure(figsize=(12,12))
    plt.scatter(data_m.price, results.resid, alpha=0.5);
    plt.hlines(0, xmax=data_m.price.max(), xmin=data_m.price.min());    

## Modelling Function
Takes in the dataframe and optional transform and multicollinearity arguments    
Checks whether to send the data to a transform function   
Selects for x_cols the features from data that are numeric, and not the price   
##### Loop

**1. Remove features with high p-values:**  
- creates the formula using x_cols     
- creates the model using formula   
- creates a dataframe, pv, of the model's p-values  
- creates a new x_cols list from the features in pv with low p-values   

**2. Handle multicollinearity:**     
- Checks whether a function has been passed for handling multicollinearity and calls accordingly    
    
         
            
            
Checks whether the length of x_cols has changed. Loop runs again if it has.     
    
    
When x_cols no longer changes, the final model and x_cols are returned





In [9]:
def modelling(datam, selector=simple_selector, transform=None, multicoll=None, 
              alpha=0.05, 
              multicollinearity_threshold = 0.7):
    
    if transform:
        datam = transform(datam)
        
    outcome = 'price'
    x_cols = [col for col in (datam.drop([outcome], axis=1).columns) 
            if isinstance(datam[col][0], numbers.Number)]
    i = 0
    while True:
        length0 = len(x_cols)
        print(f'Loop:{i}\nNumber of features: {length0}')
        
        x_cols, pvalues = selector(datam, x_cols, alpha)
        length1 = length0-len(x_cols)
        print(f'Removed by Selector: {length1}')
        
        if not i and multicoll:
            x_cols = multicoll(datam, x_cols, pvalues, 
                                      multicollinearity_threshold)
            length2 = length0 - length1 - len(x_cols)
            print(f'Removed for multicollinearity: {length2}')
        
        
        i+=1
        if len(x_cols) == length0:
            break
        
    predictors = '+'.join(x_cols)
    f = 'price' + '~' + predictors
    results = ols(formula=f, data=datam).fit()

    metrics(datam, results, x_cols)
    
    return results, x_cols

In [10]:
data_t = None


#  Model

Initial model using only cleaned data:

In [11]:
data = pd.read_csv('data/clean.csv')
#data = data[data.price < 2000000]

data.sort_index(axis=1, inplace=True)
data_train, data_test = tts(data, train_size=0.8, random_state=111)
data_train = data_train.reset_index().drop('index', axis=1)

multicollinearity_threshold=0.7
alpha=0.05


data_t = transform(data_train, poly='all')

outcome = 'price'
x_cols = [col for col in (data_t.drop([outcome], axis=1).columns) 
                    if isinstance(data_t[col][0], numbers.Number)]

x_cols, pvalues = stepwise_selector(data_t, x_cols, alpha=alpha)

x_cols = multicoll_remove(data_t, x_cols, pvalues, multicollinearity_threshold)
x_cols, pvalues = stepwise_selector(data_t, x_cols)

predictors = '+'.join(x_cols)
f = 'price' + '~' + predictors
results = ols(formula=f, data=data_t).fit()

metrics(data_t, results, x_cols)
results.summary()



ValueError: could not convert string to float: '2014-10-01'

In [ ]:
results.summary()

In [ ]:
len(data)

In [ ]:
len(results.resid)

In [ ]:
fig = sm.graphics.qqplot(results.resid, dist=stats.norm, line='45',
                         fit=True, alpha=0.1)

fig = plt.figure(figsize=(12,12))
plt.scatter(data.price, results.resid, alpha=0.5);
plt.hlines(0, xmax=data.price.max(), xmin=data.price.min());    

In [ ]:
data_t

In [12]:
data = pd.read_csv('data/clean.csv')
#data = data[data.price < 2000000]

data.sort_index(axis=1, inplace=True)
data_train, data_test = tts(data, train_size=0.8, random_state=111)
multicollinearity_threshold=0.7

#data_t = data.copy()
# data_t = transform(data_train)
data_train = data_train.reset_index().drop('index', axis=1)
data_train

,bathroomsx4,bedrooms,condition,date,floorsx2,grade,lat,long,price,sqft_above,sqft_basement,sqft_living,sqft_living15,sqft_lot,sqft_lot15,view,waterfront,yr_built,yr_renovated,zipcode
0,9,3,4,2014-10-01,4,8,47.6170,-122.288,1330000.0,2080,280,2360,2840,5504,5470,0,0,1913,1913,98122
1,12,5,3,2014-08-27,4,10,47.6518,-122.160,900000.0,2950,1400,4350,3280,37169,41631,0,0,1972,1972,98005
2,4,2,3,2014-06-12,3,7,47.6415,-122.303,620000.0,1300,130,1430,1750,3000,4000,0,0,1929,1929,98112
3,10,4,3,2015-03-31,4,9,47.3410,-122.179,415000.0,2544,0,2544,2358,4071,4179,0,0,2013,2013,98030
4,4,2,3,2015-04-29,4,7,47.6972,-122.025,300000.0,1290,0,1290,1290,2482,2482,0,0,2008,2008,98053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16982,9,5,5,2014-12-09,4,7,47.6757,-122.309,678940.0,1750,860,2610,2160,4080,4080,0,0,1909,1909,98115
16983,10,3,3,2014-05-16,4,9,47.5458,-121.996,619500.0,2170,0,2170,2170,5000,5000,0,0,2003,2003,98029
16984,6,3,3,2015-04-22,2,7,47.4729,-122.301,278000.0,1120,780,1900,1900,9994,9994,0,0,1960,1960,98168
16985,10,4,3,2014-11-18,4,10,47.5598,-122.018,645500.0,2390,0,2390,2630,9638,9258,0,0,1988,1988,98029
